# Vítej!

Tento dokument obsahuje kód k vytváření základní animace.

In [1]:
from manim import *

Manim Community v0.17.3

## Vlastní animace

In [17]:
%%manim -v WARNING -qh Intro

        
class Intro(Scene):
    def construct(self):
        square = Square(color=BLUE, fill_opacity=1).shift(LEFT * 1.5)
        circle = Circle().shift(RIGHT * 1.5)

        # 0. dostaň objekty na scénu
        self.play(FadeIn(square), Create(circle))

        # 1. animate syntax
        self.play(
            square.animate.shift(UP).scale(0.5),
            circle.animate.set_fill(GREEN, 0.75).set_stroke_color(GREEN).set_stroke_width(20),
        )

        # 2. pozicování
        self.play(square.animate.next_to(circle, LEFT, buff=0.5))
        self.play(square.animate.move_to(circle))

In [24]:
%%manim -v WARNING -qh TextScene

        
class TextScene(Scene):
    def construct(self):
        text = Text("Ahoj, tohle je fakt cool!")

        self.play(Write(text))

        tex = Tex("Ahoj, cool vzoreček: $\sum_{i = 1}^{\infty} i^2$.")
        tex.next_to(text, DOWN)

        self.play(Write(tex))

In [26]:
%%manim -v WARNING -qh GroupScene

        
class GroupScene(Scene):
    def construct(self):
        square = Square().shift(LEFT * 1.5)
        circle = Circle().shift(RIGHT * 1.5)

        group = VGroup(square, circle)

        self.play(FadeIn(square, circle))

        self.play(group.animate.scale(1.5).set_color(GREEN).rotate(PI / 12))

In [35]:
%%manim -v WARNING -qh LargeGroupScene

        
class LargeGroupScene(Scene):
    def construct(self):
        squares = VGroup()

        for i in range(100):
            squares.add(Square())

        squares.arrange_in_grid(rows=10, buff=0).scale(2/10)
        
        self.play(FadeIn(squares))

        self.wait(1.5)

In [37]:
%%manim -v WARNING -qh SVGScene

        
class SVGScene(Scene):
    def construct(self):
        self.camera.background_color = WHITE
        
        #img = ImageMObject("test.png")
        svg = SVGMobject("svg1.svg")
        
        self.play(FadeIn(svg))

In [39]:
%%manim -v WARNING -qh UpdaterScene

        
class UpdaterScene(Scene):
    def construct(self):
        square = Square()
        square_text = Tex("Tohle je cool čtverec.").next_to(square, UP)

        self.play(FadeIn(square, square_text))

        def text_updater(obj, dt):
            obj.next_to(square, UP)

        square_text.add_updater(text_updater)

        self.play(square.animate.shift(LEFT).scale(0.5).rotate(PI / 2))
        self.play(square.animate.shift(RIGHT * 3))

In [41]:
%%manim -v WARNING -qh CameraScene

        
class CameraScene(MovingCameraScene):
    def construct(self):
        triangle = Triangle(fill_opacity=1)

        self.play(FadeIn(triangle))

        self.play(self.camera.frame.animate.scale(0.25))

In [60]:
%%manim -v WARNING -qh GraphScene

        
class GraphScene(MovingCameraScene):
    def construct(self):
        graph = Graph(
            [1, 2, 3, 4, 5, 6],
            [
                (1, 2), (1, 3),
                (2, 3), (2, 4),
                (3, 4),
                (4, 5), (4, 6),
            ],
            layout_config={"seed": 0},
        )

        for vertex in graph.vertices:
            graph.vertices[vertex].scale(2)

        graph.scale(2)

        self.play(Write(graph))

        start = 1
        self.play(
            *[
                graph.vertices[i].animate.set_color(DARK_GRAY)
                for i in graph.vertices
                if i != start
            ],
            *[
                graph.edges[e].animate.set_color(DARK_GRAY)
                for e in graph.edges
            ]
        )

        def neighbours(graph, vertex):
            n = []
            for u, v in graph.edges:
                if vertex == u:
                    n.append(v)
                elif vertex == v:
                    n.append(u)
            return n

        def fix_edge(graph, e):
            if e not in graph.edges:
                e = (e[1], e[0])
            return e

        queue = [start]
        explored = {start}
        while len(queue) != 0:
            current = queue.pop(0)
            self.play(
                graph.vertices[current].animate.set_color(GREEN),
                self.camera.frame.animate.move_to(graph.vertices[current]),
            )

            newly_added_neighbours = []
            for neighbour in neighbours(graph, current):
                if neighbour in explored:
                    continue

                explored.add(neighbour)
                queue.append(neighbour)
                newly_added_neighbours.append(neighbour)

            if len(newly_added_neighbours) != 0:
                self.play(
                    *[
                        graph.vertices[n].animate.set_color(WHITE)
                        for n in newly_added_neighbours
                    ],
                    *[
                        graph.edges[fix_edge(graph, (current, n))].animate.set_color(WHITE)
                        for n in newly_added_neighbours
                    ],
                    
                )

In [61]:
%%manim -v WARNING -qh SaveRestoreScene

        
class SaveRestoreScene(MovingCameraScene):
    def construct(self):
        triangle = Triangle(fill_opacity=1)

        self.play(FadeIn(triangle))

        triangle.save_state()

        for i in range(2):
            self.play(triangle.animate.scale(1.5).set_color(GREEN).shift(DOWN).rotate(-PI / 12))

        self.play(triangle.animate.restore())        